In [2]:
import re 
import pandas as pd
import functions.currencyconverter as cc
from functions.database import createOfficeDoc, connectCollection

In [3]:
#Establish connection to database
db, coll = connectCollection('companies','companies')

In [7]:
#Create a copy of the initial collection with fields of interest only
companies = db['companies']

#Clearing 
#1- Those companies whose deadpooled_year is not null are understood as available for renting offices
isAvailable = (lambda d: True if d['deadpooled_year'] not in ['',None] else False)
for company in companies.find():
    companies.update_one(company,{'$set':{'isAvailable':isAvailable(company)}})

#2- Clean those that dont have offices, have not raised any money and do not provide founded_year information
count = companies.delete_many({'offices':[],'total_money_raised':'$0','founded_year':None})
print(count.deleted_count, ' documents deleted.')

3049  documents deleted.


In [9]:
#WEB SCRAPPING FOR CURRENCY EXCHANGE
import currencyconverter as cc

foi = {'name':1,'category_code':1,'founded_year':1,'deadpooled_year':1,'description':1,
       'total_money_raised':1,'offices':1,'isAvailable':1,'isTech':1}

df = pd.DataFrame(list(db['companies'].find({},foi)))
df['total_money_raised'] = df['total_money_raised'].apply(cc.moneyConverter)

#To take tech startups we check tech in the category_name column
isTech = lambda s: bool(re.findall('tech',s)) if s else False

df['isTech'] = (df['founded_year']>2005) & (df['total_money_raised']>1) & (df['category_code'].apply(isTech)
                                                                          & (df['isAvailable'] == False))

print('Number of tech startups availables in the dataset: ',df['isTech'].sum())
print('Number of companies with available offices in the dataset: ',df['isAvailable'].sum())

Number of tech startups availables in the dataset:  105
Number of companies with available offices in the dataset:  894


In [10]:
#Save dataframe with implemented changes in mongo
db['companies'].drop()
db['companies'].insert_many(df.T.to_dict().values())


In [12]:
#Create offices collection to save the offices information 
#This collection will be relationed with the companies collection by company_id

for company in db['companies'].find():
    for office in company['offices']:
        db['offices'].insert_one(createOfficeDoc(company,office))       
        
#Delete offices information from companies collection
db['companies'].update_many({},{'$unset':{'offices':''}})